In [1]:
import pandas as pd
import numpy as np

In [48]:
LOC = "~/Work/Data/FondAsp"
df = pd.read_csv(f"{LOC}/sat_results.csv")
df.loc[df['domain'] == 'blocksworld-ipc08', 'domain'] = 'blocksworld'
df.loc[df['domain'] == 'blocksworld-new', 'domain'] = 'blocksworld'

df = df.loc[:, ~df.columns.str.contains('Unnamed')]
df_sat = df.query(f"solved==True")

In [49]:
other_planners = ["prp-4h", "paladinus-4h", "glucose-4h", "minisat-4h"]
#asp_planners = ["asp1-4h", "asp2-backbone-4h", "asp2-backbone-4h-single", 'asp2-4h-kb', "asp2-4h-kb-single", "asp2-4h-undo", "asp2-4h-undo-single", "asp2-4h-kb2", "asp2-4h-kb2-single"]
asp_planners = ["asp1-reg-4h", "asp2-reg-4h", "asp2-reg-kb-4h", "asp2-reg-undo-4h"]

all_planners = other_planners + asp_planners

In [50]:
df_sat.planner.unique()

array(['paladinus-4h', 'prp-4h', 'asp1-4h', 'minisat-4h', 'glucose-4h',
       'asp2-backbone-4h', 'asp2-reg-4h', 'asp2-reg-undo-4h',
       'asp2-reg-kb-4h', 'asp2-4h-undo', 'asp2-4h-kb2', 'asp1-reg-4h'],
      dtype=object)

## Compute coverage

In [55]:
df_solved = df_sat.query(f"timeout==False and memoryout==False and planner in @all_planners")

df_sat_counts = df.query("planner=='paladinus-4h'").groupby(["domain"]).count()[["instance"]]
df_sat_counts.rename(columns={"instance": "total_count"}, inplace=True)
df_sat_counts


df_solved_count = df_solved.groupby(["domain", "planner"]).count()[["instance"]].reset_index()
df_solved_count.rename(columns={"instance": "count"}, inplace=True)
df_coverage = df_solved_count.merge(df_sat_counts, left_on="domain", right_on="domain", how="inner")
df_coverage["coverage"] = round(100*df_coverage["count"]/df_coverage["total_count"],2)
df_coverage.to_csv("coverage.csv")
# df_coverage

In [56]:
df_coverage.pivot_table(index=["domain"], columns="planner", values="coverage")

planner,asp1-reg-4h,asp2-reg-4h,asp2-reg-kb-4h,asp2-reg-undo-4h,glucose-4h,minisat-4h,paladinus-4h,prp-4h
domain,,,,,,,,
acrobatics,NaN,50.00,50.0,50.00,50.00,37.50,100.00,100.00
beam-walk,NaN,NaN,NaN,NaN,27.27,18.18,81.82,100.00
blocksworld,NaN,NaN,NaN,14.29,28.57,25.71,21.43,100.00
chain-of-rooms,NaN,NaN,NaN,NaN,10.00,NaN,60.00,100.00
doors,93.33,100.00,NaN,NaN,100.00,66.67,93.33,100.00
earth_observation,NaN,NaN,NaN,NaN,25.00,15.00,25.00,100.00
elevators,46.67,46.67,NaN,46.67,46.67,46.67,46.67,100.00
faults-ipc08,NaN,NaN,NaN,NaN,69.09,50.91,85.45,100.00
first-responders-ipc08,15.07,15.07,NaN,15.07,67.12,54.79,38.36,100.00


## Note on which asp planner to use

See dictionary below

In [57]:
configs={
    "acrobatics": "asp2-reg-kb-4h",
    "beam-walk": "asp2-reg-4h",
    "blocksworld" : "asp2-reg-4h",
    "chain-of-rooms": "asp2-reg-4h",
    "doors": "asp2-reg-4h",
    "earth_observation": "asp2-reg-4h",
    "elevators": "asp2-reg-undo-4h",
    "faults-ipc08": "asp2-reg-4h",
    "first-responders-ipc08": "asp2-reg-undo-4h",
    "islands": "asp2-reg-undo-4h",
    "miner": "asp2-reg-4h",
    "spiky-tireworld": "asp2-reg-kb-4h",
    "tireworld": "asp2-reg-4h",
    "tireworld-truck": "asp2-reg-undo-4h",
    "triangle-tireworld": "asp2-reg-kb-4h",
    "zenotravel": "asp2-reg-4h"
}

### Select relevant planners for ASP2

In [58]:
dfs_asp2 = []
for _d, _p in configs.items():
    df_1 = df_sat.query(f"domain==@_d and planner==@_p")
    dfs_asp2.append(df_1)
    
df_asp2 = pd.concat(dfs_asp2)
df_asp2

,domain,instance,planner,type,solved,SAT,time,memory,timeout,memoryout,policysize
14,acrobatics,p01,asp2-reg-kb-4h,asp,True,True,1.005768,1.207031,False,False,5
24,acrobatics,p03,asp2-reg-kb-4h,asp,True,True,5.008309,1.257812,False,False,16
48,acrobatics,p02,asp2-reg-kb-4h,asp,True,True,1.022346,1.148438,False,False,8
68,acrobatics,p04,asp2-reg-kb-4h,asp,True,True,3014.090484,273.664062,False,False,32
899,doors,p06,asp2-reg-4h,asp,True,True,14.374463,1.398438,False,False,15
...,...,...,...,...,...,...,...,...,...,...,...
1120,triangle-tireworld,p01,asp2-reg-kb-4h,asp,True,True,1.038336,1.164062,False,False,8
1151,triangle-tireworld,p03,asp2-reg-kb-4h,asp,True,True,54.537869,123.566406,False,False,24
1230,triangle-tireworld,p02,asp2-reg-kb-4h,asp,True,True,3.527478,1.253906,False,False,16
1261,triangle-tireworld,p05,asp2-reg-kb-4h,asp,True,True,5938.588549,460.808594,False,False,40


In [59]:
df_asp2["planner"] = "asp2"
df_asp2.head()

,domain,instance,planner,type,solved,SAT,time,memory,timeout,memoryout,policysize
14,acrobatics,p01,asp2,asp,True,True,1.005768,1.207031,False,False,5
24,acrobatics,p03,asp2,asp,True,True,5.008309,1.257812,False,False,16
48,acrobatics,p02,asp2,asp,True,True,1.022346,1.148438,False,False,8
68,acrobatics,p04,asp2,asp,True,True,3014.090484,273.664062,False,False,32
899,doors,p06,asp2,asp,True,True,14.374463,1.398438,False,False,15


In [60]:
df_asp1 = df_sat.query(f"planner=='asp1-reg-4h'")
df_asp1["planner"] = "asp1"
df_asp1

/tmp/ipykernel_569598/2323663711.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_asp1["planner"] = "asp1"


,domain,instance,planner,type,solved,SAT,time,memory,timeout,memoryout,policysize
299,miner,p50,asp1,asp,True,True,1235.749486,1733.160156,False,False,24
311,miner,p20,asp1,asp,True,True,293.860546,864.449219,False,False,19
323,miner,p35,asp1,asp,True,True,454.118283,844.285156,False,False,18
335,miner,p48,asp1,asp,True,True,1406.045844,1876.683594,False,False,24
347,miner,p22,asp1,asp,True,True,712.485423,812.718750,False,False,26
...,...,...,...,...,...,...,...,...,...,...,...
4133,tireworld-truck,p51,asp1,asp,True,True,515.012214,136.054688,False,False,20
4143,tireworld-truck,p31,asp1,asp,True,True,24.796113,1.031250,False,False,15
4153,tireworld-truck,p37,asp1,asp,True,True,4.168633,1.031250,False,False,13
4163,tireworld-truck,p19,asp1,asp,True,True,3.548557,1.160156,False,False,14


### Other Planners

In [61]:
other_planners = ["prp-4h", "paladinus-4h", "glucose-4h", "minisat-4h"]
df_others = df_sat.query(f"planner in @other_planners")
df_others

,domain,instance,planner,type,solved,SAT,time,memory,timeout,memoryout,policysize
1,acrobatics,p06,paladinus-4h,paladinus,True,True,3.153158,82.828125,False,False,128
5,acrobatics,p06,prp-4h,prp,True,True,10.479293,11.117188,False,False,129
8,acrobatics,p01,paladinus-4h,paladinus,True,True,0.378739,1.285156,False,False,4
9,acrobatics,p01,minisat-4h,fondsat,True,True,0.294426,1.402344,False,False,4
10,acrobatics,p01,glucose-4h,fondsat,True,True,0.179494,0.003906,False,False,4
...,...,...,...,...,...,...,...,...,...,...,...
4463,earth_observation,p31,glucose-4h,fondsat,True,True,6043.410634,314.640625,False,False,37
4465,earth_observation,p31,prp-4h,prp,True,True,0.227768,0.894531,False,False,42
4471,earth_observation,p37,prp-4h,prp,True,True,1.032076,0.679688,False,False,437
4477,earth_observation,p19,prp-4h,prp,True,True,0.343312,1.121094,False,False,131


### Join and export

In [62]:
df_all = pd.concat([df_asp1, df_asp2, df_others])
df_all.to_csv("./solved_results.csv")

### Compute coverage of selected planners

In [65]:

df_solved_count = df_all.groupby(["domain", "planner"]).count()[["instance"]].reset_index()
df_solved_count.rename(columns={"instance": "count"}, inplace=True)
df_coverage = df_solved_count.merge(df_sat_counts, left_on="domain", right_on="domain", how="inner")
df_coverage["coverage"] = round(100*df_coverage["count"]/df_coverage["total_count"],2)
df_coverage.to_csv(f"coverage.csv")
df_coverage

,domain,planner,count,total_count,coverage
0,acrobatics,asp2,4,8,50.00
1,acrobatics,glucose-4h,4,8,50.00
2,acrobatics,minisat-4h,3,8,37.50
3,acrobatics,paladinus-4h,8,8,100.00
4,acrobatics,prp-4h,8,8,100.00
...,...,...,...,...,...
74,triangle-tireworld,prp-4h,40,40,100.00
75,zenotravel,glucose-4h,5,15,33.33
76,zenotravel,minisat-4h,5,15,33.33
77,zenotravel,paladinus-4h,5,15,33.33


### Temp to randomly check if computations are correct

In [64]:
df_all.query(f"domain=='acrobatics' and planner=='asp2'")

,domain,instance,planner,type,solved,SAT,time,memory,timeout,memoryout,policysize
14,acrobatics,p01,asp2,asp,True,True,1.005768,1.207031,False,False,5
24,acrobatics,p03,asp2,asp,True,True,5.008309,1.257812,False,False,16
48,acrobatics,p02,asp2,asp,True,True,1.022346,1.148438,False,False,8
68,acrobatics,p04,asp2,asp,True,True,3014.090484,273.664062,False,False,32
